In [1]:
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

/Users/tracesmith/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv("./Data/train.csv",sep=",",header=0)

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,530,20,RL,0,32668,Pave,NAW,IR1,Lvl,AllPub,...,0,NP,NF,Nfeat,0,3,2007,WD,Alloca,200624
1,809,80,RL,85,13400,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,GdWo,Nfeat,0,6,2006,WD,Normal,159950
2,1112,60,RL,80,10480,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,NF,Nfeat,0,9,2008,WD,Normal,205000
3,38,20,RL,74,8532,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,NF,Nfeat,0,10,2009,WD,Normal,153000
4,1269,50,RL,0,14100,Pave,NAW,IR1,Lvl,AllPub,...,0,NP,GdWo,Nfeat,0,5,2008,WD,Normal,381000


In [5]:
X_all = data.ix[:,:-1]
print("Explanatory Variables: {}".format(X_all.shape[1]))
y = data.ix[:,-1:]
print("Response Variable: {}".format(y.shape[1]))
print("Total Observations: {}".format(y.shape[0]))

Explanatory Variables: 80
Response Variable: 1
Total Observations: 1460


In [6]:
X_all.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [7]:
X_all.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'Pav

In [8]:
for col,row in X_all.iteritems():
    print("{}: {} -- na: {}".format(col,row.dtype,row.isnull().sum()))

Id: int64 -- na: 0
MSSubClass: int64 -- na: 0
MSZoning: object -- na: 0
LotFrontage: int64 -- na: 0
LotArea: int64 -- na: 0
Street: object -- na: 0
Alley: object -- na: 0
LotShape: object -- na: 0
LandContour: object -- na: 0
Utilities: object -- na: 0
LotConfig: object -- na: 0
LandSlope: object -- na: 0
Neighborhood: object -- na: 0
Condition1: object -- na: 0
Condition2: object -- na: 0
BldgType: object -- na: 0
HouseStyle: object -- na: 0
OverallQual: int64 -- na: 0
OverallCond: int64 -- na: 0
YearBuilt: int64 -- na: 0
YearRemodAdd: int64 -- na: 0
RoofStyle: object -- na: 0
RoofMatl: object -- na: 0
Exterior1st: object -- na: 0
Exterior2nd: object -- na: 0
MasVnrType: object -- na: 0
MasVnrArea: int64 -- na: 0
ExterQual: object -- na: 0
ExterCond: object -- na: 0
Foundation: object -- na: 0
BsmtQual: object -- na: 0
BsmtCond: object -- na: 0
BsmtExposure: object -- na: 0
BsmtFinType1: object -- na: 0
BsmtFinSF1: int64 -- na: 0
BsmtFinType2: object -- na: 0
BsmtFinSF2: int64 -- na: 

### Set Explanatory Variables as "Categorical Variabes"

In [9]:
X_all['YearRemodAdd'] = X_all['YearRemodAdd'].astype(object)
X_all['YrSold'] = X_all['YrSold'].astype(object)
X_all['MoSold'] = X_all['MoSold'].astype(object)
X_all['YearBuilt'] = X_all['YearBuilt'].astype(object)
X_all['OverallCond'] = X_all['OverallCond'].astype(object)
X_all['OverallQual'] = X_all['OverallQual'].astype(object)
X_all['MSSubClass'] = X_all['MSSubClass'].astype(object)

In [10]:
for col,row in X_all.iteritems():
    print(col,'-----',row.dtype)

Id ----- int64
MSSubClass ----- object
MSZoning ----- object
LotFrontage ----- int64
LotArea ----- int64
Street ----- object
Alley ----- object
LotShape ----- object
LandContour ----- object
Utilities ----- object
LotConfig ----- object
LandSlope ----- object
Neighborhood ----- object
Condition1 ----- object
Condition2 ----- object
BldgType ----- object
HouseStyle ----- object
OverallQual ----- object
OverallCond ----- object
YearBuilt ----- object
YearRemodAdd ----- object
RoofStyle ----- object
RoofMatl ----- object
Exterior1st ----- object
Exterior2nd ----- object
MasVnrType ----- object
MasVnrArea ----- int64
ExterQual ----- object
ExterCond ----- object
Foundation ----- object
BsmtQual ----- object
BsmtCond ----- object
BsmtExposure ----- object
BsmtFinType1 ----- object
BsmtFinSF1 ----- int64
BsmtFinType2 ----- object
BsmtFinSF2 ----- int64
BsmtUnfSF ----- int64
TotalBsmtSF ----- int64
Heating ----- object
HeatingQC ----- object
CentralAir ----- object
Electrical ----- object
1st

### Preprocess Data -- Dummy Variables

In [11]:
def preprocess_features(X):
    df_preprocess = pd.DataFrame(index=X.index) # output dataframe, initially empty
    # iterate through each column
    for colname, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=colname)  
        else: pass
        #join columns to one dataframe
        df_preprocess = df_preprocess.join(col_data)  # collect column(s) in output dataframe
    return df_preprocess

In [12]:
X_all = preprocess_features(X_all)  
df_ = pd.concat([X_all,y],axis=1) #join the two df's
df_.head()# check everything looks correct

,Id,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,530,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,200624
1,809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,159950
2,1112,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,205000
3,38,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,153000
4,1269,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,381000


In [94]:
df_.to_csv("Data/train_dummy.csv") #outout csv file

In [13]:
df_.shape[1]

619

In [95]:
### Split Dataframe into 70% train 30% testing
from sklearn.model_selection import train_test_split
def test_train_split(X_all,y):
    X_train,X_test,y_train,y_test = train_test_split(X_all,y,test_size=0.30,random_state=42)
    return(X_train,X_test,y_train,y_test)

In [96]:
X_train,X_test,y_train, y_test=test_train_split(X_all,y)

In [97]:
#Training (70%)
training_ = pd.concat([X_train,y_train],axis=1)
training_.to_csv("Data/train_split_dummy.csv")
print(training_.shape[0])

1022


In [98]:
#Testing (30%)
testing_ = pd.concat([X_test,y_test],axis=1)
#Add column "." for predictions in SAS
testing_['prediction'] = "."
testing_.to_csv("Data/test_split_dummy.csv")

### Scale/Center Data -- (skip for now)

In [ ]:
from sklearn.preprocessing import StandardScaler
cols = [col for col in X_all.columns if col not in ['Id','MSSubClass',
                            'OverallQual','OverallCond','YearBuilt','YearRemodAdd'
                            'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',
                            'MoSold','YrSold','GarageYrBlt','YearBuilt','YearRemodAdd']]
cols2 = [col for col in X_all.columns if col in ['Id','MSSubClass',
                            'OverallQual','OverallCond','YearBuilt','YearRemodAdd'
                            'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',
                            'MoSold','YrSold','GarageYrBlt','YearBuilt','YearRemodAdd']]
df2 = X_all[cols]
df3 = X_all[cols2]

In [ ]:
col_cont = []
df_cont = pd.DataFrame(columns=col_cont)
for col,row in df2.iteritems():
    if row.dtype != object:
        col_cont.append(col_cont)
        df_cont = df_cont.append(row)
    else:
        pass
df_cont = df_cont.transpose()
df_cont.head()

In [ ]:
#df_cont_scale = df_cont.apply(lambda x: (x - np.mean(x)) / np.std(x))
df_cont_scale = df_cont.apply(lambda x: (x - np.mean(x)))

In [ ]:
col_cat = []
df_cat = pd.DataFrame(columns=col_cat)
for col,row in df2.iteritems():
    if row.dtype == object:
        col_cat.append(col_cat)
        df_cat = df_cat.append(row)
    else:
        pass

df_cat = df_cat.transpose()

In [ ]:
#y_scale = y.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
#y_scale = y.apply(lambda x: (x - np.mean(x)))

In [ ]:
df_merge = pd.concat([df3,df_cont_scale,df_cat,y],axis=1)
print(df_merge.shape[1])
df_merge.head()

In [ ]:
cols = ["Id","MSSubClass","MSZoning","LotFrontage","LotArea","Street", 
"Alley","LotShape","LandContour","Utilities","LotConfig", 
"LandSlope","Neighborhood","Condition1","Condition2","BldgType",
"HouseStyle","OverallQual","OverallCond","YearBuilt","YearRemodAdd","RoofStyle", 
"RoofMatl","Exterior1st","Exterior2nd","MasVnrType","MasVnrArea","ExterQual",
"ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1", 
"BsmtFinSF1","BsmtFinType2","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","Heating","HeatingQC", 
"CentralAir","Electrical","1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath",
"BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","KitchenQual", 
"TotRmsAbvGrd","Functional","Fireplaces","FireplaceQu","GarageType","GarageYrBlt", 
"GarageFinish","GarageCars","GarageArea","GarageQual","GarageCond","PavedDrive",
"WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea",
"PoolQC","Fence","MiscFeature","MiscVal","MoSold","YrSold","SaleType", 
"SaleCondition","SalePrice"]
df_merge = df_merge[cols]

In [ ]:
df_merge.head()

In [ ]:
df_merge.to_csv("train_center.csv")